<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Building Agentic AI Applications with LLMs</h1>
<h2><b>Exercise 2:</b> Metadata Generation</h2>
<br>

**Welcome to the second exercise!**

This is a lean exercise intended to reinforce the concepts of structured output to try and work with course material and even long-form markdown as an exercise medium. Specifically, we will consider how we can generate first realistic metadata, and then an actual jupyter notebook using the tools from our previous section.

### **Learning Objectives:**
**In this notebook, we will:**

- Consider a more involved example of structured output which could be directly applied to synthetic content (*if used responsibly*).
- Push beyond the generative priors of your LLM system to improve a longer-form document in an iterative fashion.

### **Setup**

Before doing this, let's load in our setup from the previous notebook and continue working with it as useful:

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia import ChatNVIDIA
from langchain_openai import ChatOpenAI
from functools import partial

from course_utils import chat_with_chain

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://llm_client:9000/v1")

## Minimum Viable Invocation
# print(llm.invoke("How is it going? 1 sentence response.").content)

## Back-and-forth loop
prompt = ChatPromptTemplate.from_messages([
    ("system",
         "You are a helpful instructor assistant for NVIDIA Deep Learning Institute (DLI). "
         " Please help to answer user questions about the course. The first message is your context."
         " Restart from there, and strongly rely on it as your knowledge base. Do not refer to your 'context' as 'context'."
    ),
    ("user", "<context>\n{context}</context>"),
    ("ai", "Thank you. I will not restart the conversation and will abide by the context."),
    ("placeholder", "{messages}")
])

## Am LCEL chain to pass into chat_with_generator
chat_chain = prompt | llm | StrOutputParser()

with open("simple_long_context.txt", "r") as f:
    full_context = f.read()

long_context_state = {
    "messages": [],
    "context": full_context,
}

chat = partial(chat_with_chain, chain=chat_chain)
chat(long_context_state)


[Human]: what's going on



[Agent]: You started by providing a context about NVIDIA's GTC 2025 Conference workshops. There are no specific user questions posed yet. Would you like to ask a question about any of the workshops listed, or would you like more information about a specific workshop?


[Human]: list the first 3 sessions



[Agent]: Here are the first 3 sessions listed:

1. **Harness NVIDIA’s Advanced Tools for Gen AI in Digital Health** - This training lab explores the capabilities of NVIDIA's AI tools in transforming digital health solutions.
2. **Advanced Medical AI Development with MONAI: From Interactive Annotation to Foundation Models** - This hands-on training lab demonstrates how to build end-to-end medical AI workflows using MONAI tools.
3. **Building Digital Twin Environments With OpenUSD, NVIDIA Isaac Sim, and ROS: A Hands-On Approach to Robotics Simulation** - This lab focuses on creating a digital twin environment using OpenUSD, NVIDIA Omniverse, and USD Search NIM, and simulating robots within it using NVIDIA Isaac Sim and ROS.


[Human]: 


<hr><br>

### **Part 1:** Generating Simple Metadata

In the lecture notebook, we picked up some techniques to generate data simply by asking nicely and enforcing a style. This relied on the model's priors. We noted how every model has some kinds of limitations in this regard. To make things easy, let's start out with an actual productionalizable use-case where even the 8B model shines; **Short-Form Data Extraction**.

Our dataset of workshops has a lot of natural-language descriptions and we have a website frontend that requires it to have some sort of a schema, so wouldn't it be great if we could use an LLM to initialize those values?

Well, we could define a schema to help us generate these values:

In [4]:
from pydantic import BaseModel, Field
from typing import List

class MetaCreator(BaseModel):
    short_abstract: str = Field(description=(
        "A concise, SEO-optimized summary (1-2 sentences) of the course for students."
        " Ensure accuracy and relevance without overstating the workshop's impact."
    ))
    topics_covered: List[str] = Field(description=(
        "A natural-language list of key topics, techniques, and technologies covered."
        " Should start with 'This workshop' and follow a structured listing format that lists at least 4 points."
    ))
    abstract_body: str = Field(description=(
        "A detailed expansion of the short abstract, providing more context and information."
    ))
    long_abstract: str = Field(description=(
        "An extended version of the short abstract, followed by the objectives."
        " The first paragraph should introduce the topic with a strong hook and highlight its relevance."
    ))
    objectives: List[str] = Field(description=(
        "Key learning outcomes that students will achieve, emphasizing big-picture goals rather than specific notebook content."
    ))
    outline: List[str] = Field(description=(
        "A structured sequence of key topics aligned with major course sections, providing a clear learning path."
    ))
    on_completion: str = Field(description=(
        "A brief summary of what students will be able to accomplish upon completing the workshop."
    ))
    prerequisites: List[str] = Field(description=(
        "Essential prior knowledge and skills expected from students before taking the course."
    ))

def get_schema_hint(schema):
    schema = getattr(schema, "model_json_schema", lambda: None)() or schema
    return ( # PydanticOutputParser(pydantic_object=Obj.model_schema_json()).get_format_instructions()
        'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema'
        ' {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}},'
        ' "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema.'
        ' The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n' + str(schema) + '\n```'
    )

schema_hint = get_schema_hint(MetaCreator)
# schema_hint

<br>

Then, if we just bind our LLM client to abide by the schema, then we should be able to generate it. 
The code below not only does that, but also shows how one might go about streaming the data or even filtering the data.

In [6]:
structured_llm = llm.with_structured_output(
    schema=MetaCreator.model_json_schema(), 
    strict=True
)

meta_chain = prompt | structured_llm
meta_gen_directive = (
    # f"Can you generate a course entry on the Earth-2 course? {schema_hint}"
    # f"Can you combine the topics of the Earth-2 course and the NeRF/3DGS courses and generate a compelling course entry? {schema_hint}"
    f"Can you combine the topics of the Earth-2 course and the NeRF/3DGS courses and generate a compelling course entry? Make sure to explain how they combine. {schema_hint}"
) 
meta_gen_state = {
    "messages": [("user", meta_gen_directive)],
    "context": full_context,
}

# answer = meta_chain.invoke(meta_gen_state)
# print(answer)

from IPython.display import clear_output

answer = {}
for chunk in meta_chain.stream(meta_gen_state):
    clear_output(wait=True)
    for key, value in chunk.items():
        print(f"{key}: {value}", end="\n\n", flush=True)
        answer[key] = value

llm._client.last_response.json()

short_abstract: This workshop combines the latest advances in Generative AI (Gen AI) from NVIDIA's Earth-2 platform with expert techniques for creating high-quality static and dynamic 3D scenes using Neural Radiance Fields (NeRF) and 3D Gaussian Splats (3DGS). By the end of this course, you'll be able to integrate Earth-2's AI-driven weather and climate modeling capabilities with state-of-the-art 3D scene reconstruction methods.

topics_covered: ['Earth-2 AI-driven weather and climate modeling', 'Neural Radiance Fields (NeRF) for static 3D scene reconstruction', '3D Gaussian Splats (3DGS) for dynamic 3D scene reconstruction', 'Integrating Earth-2 with NeRF/3DGS for high-quality 3D scene creation', 'Applying advanced methods for static and dynamic scene reconstruction in interactive 3D worlds']

abstract_body: In this workshop, we'll delve into the latest advancements in Earth-2, a platform that combines AI-driven weather and climate modeling with high-performance computing. We'll explo

{'object': 'list',
 'data': [{'id': '01-ai/yi-large',
   'object': 'model',
   'created': 735790403,
   'owned_by': '01-ai'},
  {'id': 'abacusai/dracarys-llama-3.1-70b-instruct',
   'object': 'model',
   'created': 735790403,
   'owned_by': 'abacusai'},
  {'id': 'ai21labs/jamba-1.5-large-instruct',
   'object': 'model',
   'created': 735790403,
   'owned_by': 'ai21labs'},
  {'id': 'ai21labs/jamba-1.5-mini-instruct',
   'object': 'model',
   'created': 735790403,
   'owned_by': 'ai21labs'},
  {'id': 'aisingapore/sea-lion-7b-instruct',
   'object': 'model',
   'created': 735790403,
   'owned_by': 'aisingapore'},
  {'id': 'baichuan-inc/baichuan2-13b-chat',
   'object': 'model',
   'created': 735790403,
   'owned_by': 'baichuan-inc'},
  {'id': 'bigcode/starcoder2-15b',
   'object': 'model',
   'created': 735790403,
   'owned_by': 'bigcode'},
  {'id': 'bigcode/starcoder2-7b',
   'object': 'model',
   'created': 735790403,
   'owned_by': 'bigcode'},
  {'id': 'bytedance/seed-oss-36b-instruct'

<br>

Ok! That's not bad! It's reflective of the same limitations that we discussed in the lecture, but it does seem to be making good use of its context (while not degenerating into nonsense). Maybe we can ask it to improve upon it?

In [ ]:
## TODO: See if you can't prompt-engineer this solution to lead to an improved autoregression.
meta_gen_state = {
    "messages": [
        ("user", meta_gen_directive),
        ("ai", str(answer)),
        ("user", "Great! Can you please correct any mistakes and flesh out some vagueness?")
    ],
    # "context": full_context,  ## Maybe we don't need the full context
    "context": "",
}

answer2 = {}
for chunk in meta_chain.stream(meta_gen_state):
    clear_output(wait=True)
    for key, value in chunk.items():
        print(f"{key}: {value}", end="\n\n", flush=True)
        answer2[key] = value

<br>

**Yeah... it can get better to a point.** 
- If we incorporate chat history, you'll start running into issues fast as the model starts to reach its context limit.
- If we don't, we can still squeeze some customization from the LLM and can reasonably generate a better or longer outline... to a point.

For this use-case, this model actually isn't that bad, but for something a bit longer, the limitations clearly start to show...

In [ ]:
## Pick your preferred option
final_answer = answer2

<hr><br>

### **Part 2:** Generating A Notebook

We've seen some fuzzy limitations when trying to generate our metadata, so let's see if we start to see more obvious problems when we get more ambitious. Below, we show an attempt at using the GPT-4o model to generate a notebook:

In [10]:
from IPython.display import display, Markdown, Latex
with open("chats/make_me_a_notebook/input.txt", "r") as f:
    notebook_input_full = f.read()
    notebook_input_prompt = notebook_input_full.split("\n\n")[-1]
print(notebook_input_full)
# print(notebook_input_prompt)

short_abstract: Accelerate AI Weather Models and 3D Visualizations with NVIDIA Earth-2, NeRFs, and Gaussian Splatting

topics_covered: This workshop combines the power of NVIDIA Earth-2, Neural Radiance Fields (NeRFs), and 3D Gaussian Splatting to accelerate AI weather modeling and 3D visualizations. You'll learn how to leverage these technologies to downscale models, generate super-resolution outputs, and speed up complex 3D rendering pipelines.

abstract_body: AI-driven climate modeling and weather forecasting rely on complex simulations that can be computationally expensive. To address this challenge, NVIDIA Earth-2 provides a scalable platform for accelerating AI workloads. This workshop focuses on three key technologies: Neural Radiance Fields (NeRFs), a technique for rendering 3D scenes from scratch; 3D Gaussian Splatting, a method for efficiently rendering large datasets; and NVIDIA Earth-2, a platform for accelerating AI workloads. By combining these technologies, participants 

In [11]:
# !cat chats/make_me_a_notebook/output.txt
display(Markdown("chats/make_me_a_notebook/output.txt"))
display(Markdown("<hr><br><br>"))

### NVIDIA DLI Course: Accelerate AI Weather Models and 3D Visualizations

# Introduction

This notebook will guide you through the key concepts of NVIDIA Earth-2, Neural Radiance Fields (NeRFs), and 3D Gaussian Splatting. By the end, you'll be able to:
- Render complex 3D scenes using NeRFs.
- Efficiently render large datasets with 3D Gaussian Splatting.
- Use NVIDIA Earth-2 to accelerate AI weather modeling.

Let's get started!

## 1. Setup Environment

```python
# Install necessary libraries (uncomment the lines if running locally)
# !pip install torch torchvision torchaudio
# !pip install numpy matplotlib opencv-python-headless

import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
```

## 2. Introduction to Neural Radiance Fields (NeRFs)

NeRFs enable the synthesis of photorealistic 3D scenes from sparse 2D images by learning a volumetric scene representation.

```python
# Sample code to define a simple NeRF-like function

def nerf_function(xyz):
    """Simple function simulating NeRF behavior."""
    return np.exp(-np.linalg.norm(xyz, axis=-1))

# Generate a sample 3D coordinate grid
xyz = np.random.rand(100, 3) * 2 - 1  # Values between -1 and 1
nerf_values = nerf_function(xyz)

# Visualizing the NeRF function output
plt.scatter(xyz[:, 0], xyz[:, 1], c=nerf_values, cmap='viridis')
plt.colorbar(label='Intensity')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('NeRF Function Output')
plt.show()
```

## 3. Introduction to 3D Gaussian Splatting

3D Gaussian Splatting allows efficient rendering of large point cloud datasets, commonly used for weather and environmental simulations.

```python
# Simulating 3D Gaussian Splatting

def gaussian_splatting(points, sigma=0.1):
    """Apply Gaussian smoothing to a 3D point cloud."""
    return np.exp(-np.linalg.norm(points, axis=-1) / (2 * sigma**2))

# Generating a synthetic 3D point cloud
points = np.random.randn(500, 3) * 0.5
splat_values = gaussian_splatting(points)

# Visualizing the Gaussian splatting effect
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=splat_values, cmap='coolwarm')
ax.set_title('3D Gaussian Splatting')
plt.show()
```

## 4. Using NVIDIA Earth-2 for AI Weather Modeling

NVIDIA Earth-2 provides acceleration for AI weather models. Below is an interactive example where you can modify weather parameters.

```python
# Simulating a simplified AI weather model
from ipywidgets import interact

def simple_weather_model(temperature, humidity, wind_speed):
    """A basic model for simulating temperature impact on climate."""
    return temperature * 0.5 + humidity * 0.3 + wind_speed * 0.2

interact(simple_weather_model, temperature=(0, 50, 5), humidity=(0, 100, 10), wind_speed=(0, 50, 5));
```

## 5. Real-World Applications

Case studies will be covered in the workshop, but here’s an example of how these methods can be applied in climate simulations.

```python
# Combining NeRFs, Gaussian Splatting, and AI weather models

def combined_model(xyz, temp, humidity, wind):
    """Simulates a weather model integrating NeRFs and Gaussian Splatting."""
    nerf_effect = nerf_function(xyz)
    splatting_effect = gaussian_splatting(xyz)
    weather_effect = simple_weather_model(temp, humidity, wind)
    return nerf_effect * splatting_effect * weather_effect

# Sample combined simulation
xyz_sample = np.random.rand(200, 3) * 2 - 1
weather_output = combined_model(xyz_sample, temp=30, humidity=60, wind=15)

# Visualization
plt.scatter(xyz_sample[:, 0], xyz_sample[:, 1], c=weather_output, cmap='plasma')
plt.colorbar(label='Simulation Output')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Combined AI Weather Model Output')
plt.show()
```

## Conclusion

In this notebook, we explored:
- NeRFs for rendering 3D scenes.
- 3D Gaussian Splatting for efficient large dataset visualization.
- NVIDIA Earth-2’s role in AI weather modeling.

Continue exploring these topics and apply them to real-world weather simulations!


<hr><br><br>

<br>

The notebook output in [`chats/make_me_a_notebook/output.txt`](./chats/make_me_a_notebook/output.txt) is the first-attempt output that came out of GPT-4o when I asked it to generate a notebook per [`chats/make_me_a_notebook/input.txt`](./chats/make_me_a_notebook/input.txt). It's serviceable enough with such a vague input, and can be improved **to some point** by just asking it for better output, criticizing it, and giving it enough information to work with. 

The common anecdote "garbage in, garbage out" comes to mind, since the LLM is just mirroring the style of reasonable output given your input specific input. But due to the conversational nature of the training (not helped by the chat prompts into which the messages are being funneled), your output will usually be uncomfortably short and just imprecise enough for many advanced use cases.

Still, let's see if we can improve on this output by giving our LLM a style reference and asking it to rephrase the notebook a bit:

In [12]:
import json

def notebook_to_markdown(path: str) -> str:
    """Load a Jupyter notebook from a given path and convert it to Markdown format."""
    with open(path, 'r', encoding='utf-8') as file:
        notebook = json.load(file)
    markdown_content = []
    for cell in notebook['cells']:
        if cell['cell_type'] == 'code':          # Combine code into one block
            markdown_content += [f'```python\n{"".join(cell["source"])}\n```']
        elif cell['cell_type'] == 'markdown':    # Directly append markdown source
            markdown_content += ["".join(cell["source"])]
        # for output in cell.get('outputs', []):   # Optionally, you can include cell outputs
        #     if output['output_type'] == 'stream':
        #         markdown_content.append(f'```\n{"".join(output["text"])}\n```')
    return '\n\n'.join(markdown_content)

notebook_example = notebook_to_markdown("extra_utils/general_representative_notebook.ipynb")

context = str(final_answer)
# context = (
#     f"THE FOLLOWING IS AN EXAMPLE NOTEBOOK FOR STYLE ONLY: \n\n{notebook_example}"
#     "\n\n=========\n\n"
#     f"THE FOLLOWING IS THE TOPIC COURSE THAT WE ARE DISCUSSING:\n\n{final_answer}\n\n"
# )

long_context_state = {
    "messages": [],
    "context": context,
}

chat = partial(chat_with_chain, chain=chat_chain)
chat(long_context_state)

## EXAMPLE INPUTS ##
## Option: Can you please construct a good notebook in markdown format?
## Option: That's great, but there is no code. Can you please flesh out each section within an end-to-end narrative?


[Human]: construct a good notebook in markdown format



[Agent]: # Comprehensive Workshop on Generative AI and 3D Scene Reconstruction

## Introduction

This workshop focuses on utilizing the NVIDIA Earth-2 platform and combining it with expert techniques in creating high-quality static and dynamic 3D scenes using Neural Radiance Fields (NeRF) and 3D Gaussian Splats (3DGS).

## Topics Covered

* Earth-2's AI-driven weather and climate modeling capabilities
* State-of-the-art 3D scene reconstruction using Neural Radiance Fields (NeRF)
* Expert techniques for 3D Gaussian Splats (3DGS) and dynamic 3D scene reconstruction
* Integrating Earth-2 with NeRF/3DGS for photorealistic 3D scene creation
* Real-time rendering and visualization of dynamic weather simulations and 3D scenes

## Objectives

* Integrate Earth-2's AI-driven weather and climate modeling with NeRF/3DGS methods
* Apply NeRF/3DGS techniques to produce high-quality static and dynamic 3D scenes
* Utilize Earth-2 to create accurate and realistic weather simulations
* Produce photore


[Human]: flesh out each section within an end-to-end narrative including code chunks



[Agent]: # Comprehensive Workshop on Generative AI and 3D Scene Reconstruction

## Introduction

Welcome to our comprehensive workshop on generative AI and 3D scene reconstruction! In this workshop, we will be focusing on utilizing the NVIDIA Earth-2 platform and combining it with expert techniques in creating high-quality static and dynamic 3D scenes using Neural Radiance Fields (NeRF) and 3D Gaussian Splats (3DGS). By the end of this workshop, you will have a hands-on understanding of how to integrate Earth-2's AI-driven weather and climate modeling capabilities with state-of-the-art 3D scene reconstruction methods.

## Topics Covered

### Earth-2's AI-driven Weather and Climate Modeling Capabilities

In this section, we will be covering the basics of Earth-2 and its capabilities for creating photorealistic weather simulations. We will explore the different components of Earth-2, including its AI-driven weather and climate modeling capabilities, and learn how to create realistic wea


[Human]: stop



[Agent]: I'll stop providing the code chunks and narrative for the workshop materials. If you have any further requests or questions, feel free to ask.


[Human]: 


<br>

In our case, our model is quite small and we're also limiting our endpoint to a short input and short output (for its own good), so the amount of content it can generate really is quite limited. This limitation does, however, manifest in all realistic scenarios regardless of the model quality. For any modern LLM:
- Though straight decoding of the solution can work for some contexts, they cannot scale up to arbitrarily-large inputs or outputs. 
- The quality output length is generally shorter than the quality input length when we get to longer sequences. This is enforced during training and enforces good properties for efficient cost of generation and reduction in context accumulation.

In other words, **the space of things that can be given to or expected of an LLM $>>$ the space of things that an LLM can actually understand well $>>$ the space of things that the LLM can actually output well.** *($>>$ = "far greater than")*

Given this insight, we can understand that trying to force the LLM to produce a notebook all at once might lead to incoherence at the global scale. However, it seems to be starting off at least somewhat ok, so maybe there's some merit in the approach.

<hr><br>

### **Part 3:** Using an Agent Canvas

When we observe that we can't directly output the thing that we want, the next question is "can we take in what we want." 
- It seemed like the LLM was able to roughly follow along with the premise when we only gave it the premise as input, but started derailing when we gave it a representative example. 
- Furthermore, it was likely able to actually improve upon your notebook through conversation, so maybe we can start there.

**Canvasing Approach:** Instead of getting the model to predict the full document, get it to treat the document as  an environment and propose one of the following to the LLM:
> - ***"Please propose a modification that will improve the state of the document. Here are your options. Pick one/several and they will be done."***
> - ***"Here is the whole state, and you are tasked with improving JUST THIS SUBSECTION OF IT. Please output your update to that section. No other sections will be modified."***
> - ***"This is the whole document. This section is bad because of one or more of the following: {criticisms}. Replace it with an improved version."***

If the model is capable of understanding both the full environment and the instruction, then it can directly autoregress only a small section or even a strategic modification of the output. Combine this approach with structured output or chain of thought, and you're likely to get a formulation that, while not perfect, helps to approach the potential output length towards the potential input length of the model.

In [16]:
## TODO: Insert a notebook of choice
STARTING_NOTEBOOK = """
""".strip()

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system",
         "You are a helpful instructor assistant for NVIDIA Deep Learning Institute (DLI). "
         " Please help to answer user questions about the course. The first message is your context."
         " Restart from there, and strongly rely on it as your knowledge base. Do not refer to your 'context' as 'context'."
    ),
    ("user", "<context>\n{context}</context>"),
    ("ai", "Thank you. I will not restart the conversation and will abide by the context."),
    ("user", (
        "The following section needs some improvements:\n\n<<<SECTION>>>\n\n{section}\n\n<<</SECTION>>>\n\n"
        "Please propose an upgrade that would improve the overall notebook quality."
        " Later sections will follow and will be adapted by other efforts."
        " You may only output modifications to the section provided here, no later or earlier sections."
        " Follow best style practices, and assume the sections before this one are more enforcing that the latter ones."
        " Make sure to number your section, continuing from the previous ones."
    )),
])

## An LCEL chain to pass into chat_with_generator
sub_chain = prompt | llm | StrOutputParser()

delimiter = "###"  ## TODO: Pick a delimiter that works for your notebook
WORKING_NOTEBOOK = STARTING_NOTEBOOK.split(delimiter)
output = ""
for i in range(len(WORKING_NOTEBOOK)):
    chunk = WORKING_NOTEBOOK[i]
    ## TODO: Knowing that the state needs "context" and "section" values,
    ## can you construct your input state?
    chunk_refinement_state = {
    "context": "####".join(WORKING_NOTEBOOK),
    "section": chunk,
}
    for token in sub_chain.stream(chunk_refinement_state):
        print(token, end="", flush=True)
        output += token
    WORKING_NOTEBOOK[i] = output
    print("\n\n" + "#" * 64 + "\n\n")

**3. Topics Covered**

In this section, we will provide a detailed overview of the topics covered in the workshop, including code examples to help solidify your understanding.

**3.1 Earth-2's AI-driven Weather and Climate Modeling Capabilities**

Earth-2 is a powerful platform developed by NVIDIA that enables the simulation of realistic weather and climate scenarios. We will explore the capabilities of Earth-2, including its AI-driven weather and climate modeling capabilities, and learn how to create accurate and realistic weather simulations using Earth-2.

```python
import earth2 as e2

# Create a new Earth-2 object
earth = e2.Earth2()

# Set the location and time of the simulation
location = [40.7128, -74.0060]  # coordinates for New York City
time = 12  # 12:00 PM

# Create a weather simulation using Earth-2
weather_sim = earth.simulate_weather(location, time)

# Print the resulting weather simulation
print(weather_sim)
```

**3.2 State-of-the-art 3D Scene Reconstruction using Neu

<br>

<details><summary><b>Solution</b></summary>

```python
chunk_refinement_state = {
    "context": "####".join(WORKING_NOTEBOOK),
    "section": chunk,
}
```
    
</details>

<hr><br>

### **Part 4:** Reflecting On This Exercise

As we can see, this approach is quite promising in that it's able to extend the output of the model towards a large context with only local modifications. This 8B model was pretty quickly pushed out of its training distribution with this approach, and it also likely started to go pretty aggressively into hallucination mode due to its vague inputs, but a larger model would be able to iterate on this process for much longer and could even have some error-correcting or randomization efforts thrown in to stabilize the process. 

This technique is also used in the wild to implement features like codebase modification and collaborative document editing (i.e. OpenAI Canvas). Additionally, even minor modifications to this approach can help you implement some surprisingly-effective and efficient solutions:
- **Find-Replace Canvas:** Instead of autoregressing the sections of a document, you can generate find-replace pairs. Executing this process on the chunks, you will wind up with a much safer formulation as well as an easier-to-track footprint. This kind of system can be used to implement AI-enabled spell-checkers and other forms or strategic error correction.
- **Document Translation:** More generally, this approach can also be used to translate a document, one section at a time, from one format to another. A similar approach to the one above can be used to translate a document from one language to another, with a bit of context injection thrown in to help give the translating model pipeline some style to guide it.

Note that while we call this process *"canvasing,"* you may also run into the same or similar idea under the term *"iterative refinement."* They are pretty much one-and-the-same, except the latter is much more general and could technically be applied to any LLM-enabled loop that progresses the input into the output over many iterations. Canvasing implies more strongly that you're using the current environment as a playground and can make strategic modifications to improve the state.

----

In any case, we've now tested out how our little model can actually help us do some surprisingly-interesting things, while also reflecting on the fact that it has clear limitations. This marks the end of our "simple pipeline" exercises for this course. In the next section, we will be using the primitives we've picked up to start working with a proper agents framework while sticking to our very-limited but surprisingly-flexible Llama-8B model.

<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>